In [61]:
from ortools.sat.python import cp_model

model = cp_model.CpModel()

horizon = 24 

jobs = [
    {"duration": 1},                
    {"fixed_start": 8 , "duration": 1}, 
    {"fixed_start": 6 , "duration": 1},
    {"preferred_start": 6 , "duration": 1}, 
    {"preferred_start": 7 , "duration": 1} 
]
job_variables = {}
penalty_terms = []

#print(list(enumerate(jobs)))

for i, job in enumerate(jobs):
  if 'fixed_start' in job:
    start_time = model.NewIntVar(job['fixed_start'],job['fixed_start'],f"job{i}_start")
    #print(job['fixed_start'])
  elif 'preferred_start' in job:
    start_time = model.NewIntVar(0,horizon - job['duration'], f"job{i}_start")
    preferred_start = job['preferred_start']
    deviation = model.NewIntVar(0, horizon, f"deviation_job{i}")
    model.AddAbsEquality(deviation, start_time - preferred_start)
    penalty_terms.append(deviation)
    print(penalty_terms,deviation)
  else:
    start_time = model.NewIntVar(0,horizon - job['duration'], f"job{i}_start")
  job_variables[i] = {
    "start_time": start_time,
    "duration" : job['duration']
    }
# job_end = [model.new_int_var(job_preference[i], job_preference[i]+duration, f"JobEnd{i}") for i in range (number_of_jobs)]
# model.add_all_different(job_end)
# print(job_end)

# print(fixed_start_jobs,preferred_start_jobs)
# print(fixed_start_times,preferred_start_times)
#print(job_variables)

intervals = []
for i, job in enumerate(jobs):
    start_time = job_variables[i]['start_time']
    duration = job_variables[i]['duration']
    end_time = start_time + duration
    intervals.append(model.NewIntervalVar(start_time, duration, end_time, f"Job{i}_interval"))

model.AddNoOverlap(intervals)

makespan = model.NewIntVar(0 , horizon , 'makespan')

job_end_times = [job_variables[i]['start_time'] + job_variables[i]['duration'] for i in range(len(jobs))]

# for fixed_job in fixed_start_jobs:
#         for preferred_job in preferred_start_jobs:
#             model.Add(job_vars[fixed_job]['start'] <= job_vars[preferred_job]['start'])
#print(job_variables)

# for i,j in enumerate(fixed_start_times):
#     for n,m in enumerate(preferred_start_times):
#        if j == m:
#           print(i,j,n,m,fixed_start_jobs[i],preferred_start_jobs[n])
#           print(job_variables[fixed_start_jobs[i]]['start_time'],job_variables[preferred_start_jobs[n]]['start_time'])
#           model.Add(job_variables[fixed_start_jobs[i]]['start_time'] <= job_variables[preferred_start_jobs[n]]['start_time'])
# fixed_start_jobs = [i for i, job in enumerate(jobs) if 'fixed_start' in job]
# preferred_start_jobs = [i for i, job in enumerate(jobs) if 'preferred_start' in job]

# fixed_start_times = [job['fixed_start'] for job in jobs if 'fixed_start' in job]
# preferred_start_times = [job['preferred_start'] for job in jobs if 'preferred_start' in job]

# 

# for i in fixed_start_jobs:
#     for j in preferred_start_jobs:
#         fixed_start = job_variables[i]['start_time']
#         preferred_start = job_variables[j]['start_time']
#         print(fixed_start,preferred_start)
#         print(i,j)
#         print(jobs[i]["fixed_start"],jobs[j]["preferred_start"])
#         if jobs[i]["fixed_start"] == jobs[j]["preferred_start"]:
#           model.Add(fixed_start != preferred_start)
#         else:
#           preferred_start_temp = model.NewIntVar(jobs[j]['preferred_start'],jobs[j]['preferred_start'], "temp")
#           model.Add(preferred_start == preferred_start_temp)

model.AddMaxEquality(makespan, job_end_times)

model.Minimize(makespan + sum(penalty_terms))

solver = cp_model.CpSolver()
status = solver.Solve(model)

if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print(f"schedule found {solver.ObjectiveValue()} hours")
    for i, job in enumerate(jobs):
        print(f"Job {i + 1} starts at {solver.Value(job_variables[i]['start_time'])} ")
else:
    print("No solution found.")





[deviation_job3(0..24)] deviation_job3
[deviation_job3(0..24), deviation_job4(0..24)] deviation_job4
schedule found 10.0 hours
Job 1 starts at 0 
Job 2 starts at 8 
Job 3 starts at 6 
Job 4 starts at 5 
Job 5 starts at 7 


In [52]:
jobs = [
    {"duration": 1},                
    {"fixed_start": 8 , "duration": 1}, 
    {"fixed_start": 6 , "duration": 1},
    {"preferred_start": 6 , "duration": 1}, 
    {"preferred_start": 7 , "duration": 1} 
]
print(jobs[2]["fixed_start"])

6
